<a href="https://colab.research.google.com/github/LuisFerRosas/nuevo_proyecto/blob/main/nuevo__modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Colab Notebooks/Proyecto_ia3

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/Colab Notebooks/Proyecto_ia3


In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch import Tensor 
import math
import json

In [ ]:
!ls

 checkpoint			    'nuevo_ modelo.ipynb'
'Copia de Final Transformer.ipynb'  'nuevo_ modelo_local.ipynb'
'Copia de nuevo_ modelo.ipynb'	     nuevo_proyecto
 datos2				     transcripcion_de_Musica
'Final Transformer.ipynb'


In [ ]:
with open("./datos2/vocabulario2.json", "r") as fp:
        vocabPart = json.load(fp)

len(vocabPart)

233

In [ ]:
!git clone https://github.com/LuisFerRosas/nuevo_proyecto.git

fatal: destination path 'nuevo_proyecto' already exists and is not an empty directory.


In [ ]:
cd nuevo_proyecto/

/gdrive/My Drive/Colab Notebooks/Proyecto_ia3/nuevo_proyecto


In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 124 kB 14.7 MB/s 


In [ ]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)


class PositionalEncoding(nn.Module):
  def __init__(self,
                emb_size: int,
                dropout: float,
                maxlen: int = 6000):
      super(PositionalEncoding, self).__init__()
      den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
      pos = torch.arange(0, maxlen).reshape(maxlen, 1)
      pos_embedding = torch.zeros((maxlen, emb_size))
      pos_embedding[:, 0::2] = torch.sin(pos * den)
      pos_embedding[:, 1::2] = torch.cos(pos * den)
      pos_embedding = pos_embedding.unsqueeze(-2)

      self.dropout = nn.Dropout(dropout)
      self.register_buffer('pos_embedding', pos_embedding)

  def forward(self, token_embedding: Tensor):
      return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

In [ ]:
class TmusicTrasforms(nn.Module):
  def __init__(self,tgt_vocabulario,emb=40):
    super().__init__() 
    
    ################ TRANSFORMER BLOCK #############################
    # maxpool the input feature map/tensor to the transformer 
    # a rectangular kernel worked better here for the rectangular input spectrogram feature map/tensor
    self.transformer_maxpool = nn.MaxPool2d(kernel_size=[1,4], stride=[1,4])
    
    
    self.tgt_tok_emb = TokenEmbedding(tgt_vocabulario, emb)
    self.positional_encoding = PositionalEncoding(emb, dropout=0.1)
    
    transformer_layer = nn.TransformerEncoderLayer(
        d_model=40, # input feature (frequency) dim after maxpooling 40*1723 -> 40*430 (MFC*time)
        nhead=8, # 4 self-attention layers in each multi-head self-attention layer in each encoder block
        dim_feedforward=512, # 2 linear layers in each encoder block's feedforward network: dim 40-->512--->40
        dropout=0.4, 
        activation='relu' # ReLU: avoid saturation/tame gradient/reduce compute time
    )
    
    self.transformer_encoder = nn.TransformerEncoder(transformer_layer, num_layers=6)
    
    transformer_decoder_layer=nn.TransformerDecoderLayer(
        d_model=emb,
        nhead=8,
        dim_feedforward=512,
        dropout=0.4,
        activation='relu'
    )
    self.transformer_decoder=nn.TransformerDecoder(transformer_decoder_layer,num_layers=6)

    ############### 1ST PARALLEL 2D CONVOLUTION BLOCK ############
    # 3 sequential conv2D layers:  (1,40,1723) --> (8, 20, 861) -> (16, 4, 172) -> (32, 2, 86) -> (64,1,43)
    self.conv2Dblock1 = nn.Sequential(
        
        # 1st 2D convolution layer
        nn.Conv2d(
            in_channels=1, # input volume depth == input channel dim == 1
            out_channels=2, # expand output feature map volume's depth to 16
            kernel_size=3, # typical 3*3 stride 1 kernel
            stride=1,
            padding=1,
            
                  ),
        nn.BatchNorm2d(2), # batch normalize the output feature map before activation
        nn.ReLU(), # feature map --> activation map
        nn.MaxPool2d(kernel_size=2, stride=2), #typical maxpool kernel size
        nn.Dropout(p=0.3), #randomly zero 30% of 1st layer's output feature map in training
        
        # 2nd 2D convolution layer identical to last except output dim, maxpool kernel
        nn.Conv2d(
            in_channels=2, 
            out_channels=4, # expand output feature map volume's depth to 32
            kernel_size=3,
            stride=1,
            padding=1,
            
                  ),
        nn.BatchNorm2d(4),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2), # increase maxpool kernel for subsequent filters
        nn.Dropout(p=0.3), 

       
    )
    ############### 2ND PARALLEL 2D CONVOLUTION BLOCK ############
    # 3 sequential conv2D layers: (1,40,1723) --> (8, 20, 861) -> (16, 4, 172) -> (32, 2, 86) -> (64,1,43)
    self.conv2Dblock2 = nn.Sequential(
       # 1st 2D convolution layer
        nn.Conv2d(
            in_channels=1, # input volume depth == input channel dim == 1
            out_channels=2, # expand output feature map volume's depth to 16
            kernel_size=3, # typical 3*3 stride 1 kernel
            stride=1,
            padding=1,
            
                  ),
        nn.BatchNorm2d(2), # batch normalize the output feature map before activation
        nn.ReLU(), # feature map --> activation map
        nn.MaxPool2d(kernel_size=2, stride=2), #typical maxpool kernel size
        nn.Dropout(p=0.3), #randomly zero 30% of 1st layer's output feature map in training
        
        # 2nd 2D convolution layer identical to last except output dim, maxpool kernel
        nn.Conv2d(
            in_channels=2, 
            out_channels=4, # expand output feature map volume's depth to 32
            kernel_size=3,
            stride=1,
            padding=1,
            
                  ),
        nn.BatchNorm2d(4),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2), # increase maxpool kernel for subsequent filters
        nn.Dropout(p=0.3), 
       
    )

   
    
    self.fc2_linear = nn.Linear(emb,tgt_vocabulario)
    
    
    self.softmax_out = nn.Softmax(dim=2) # dim==1 is the freq embedding
  
  def forward(self,x,partitura_tok,wave_comple):
    # print("x input : "+str(x.shape))# (1,1,40,1723)
    ############ 1st parallel Conv2D block: 4 Convolutional layers ############################
    
    conv2d_embedding1 = self.conv2Dblock1(x) # x == N/batch * channel * freq * time
    # print("conv2d_embedding1 : "+str(conv2d_embedding1.shape))# [N, 4, 10, 215]
    
    conv2d_embedding1 = conv2d_embedding1.reshape(conv2d_embedding1.shape[0],-1,conv2d_embedding1.shape[-1]) #[n,40,215]
    # print("conv2d_embedding1 Reshape : "+str(conv2d_embedding1.shape))#[n,40,215]
    ############ 2nd parallel Conv2D block: 4 Convolutional layers #############################
    
    conv2d_embedding2 = self.conv2Dblock2(x) # x == N/batch * channel * freq * time
    # print("conv2d_embedding2 : "+str(conv2d_embedding2.shape))# [N, 4, 10, 215]
   
    conv2d_embedding2 = conv2d_embedding2.reshape(conv2d_embedding2.shape[0],-1,conv2d_embedding2.shape[-1])#[n,40,215]
    # print("conv2d_embedding2 Reshape : "+str(conv2d_embedding2.shape))#[n,40,215]
   
    # x_maxpool = self.transformer_maxpool(x)
    # print("x_maxpool  : "+str(x_maxpool.shape))#[1, 1, 40, 430]
    
    x_maxpool_reduced = wave_comple
    print("wave 2 : "+str(x_maxpool_reduced.shape))#[1, 40, 581]
    
    wave_comple = wave_comple.permute(2,0,1) 
    print("x------>entrada Encoder : "+str(wave_comple.shape))#[581, 1, 40]

    transformer_output = self.transformer_encoder(wave_comple)
    print("salida  Encoder : "+str(transformer_output.shape))#[581, 1, 40]
    transformer_output=transformer_output.permute(1,2,0) #[1, 40, 581] 
    
    
    sumaTensores=torch.cat((conv2d_embedding1, conv2d_embedding2, transformer_output), 2)
    print("sumaTensores : "+str(sumaTensores.shape))#[1, 40, 1011]
    entrada_Decoder=sumaTensores.permute(2,0,1)#[430,n,40]

    print("entrada_Decoder : "+str(entrada_Decoder.shape))#[1011, 1, 40]
    memory=self.positional_encoding(entrada_Decoder)
    # print("memory positional  : "+str(memory.shape))#[1011, 1, 40]

    partitura_tok=partitura_tok.transpose(1,0)#[1,45]
    # print("partitura_tok  : "+str(partitura_tok.shape))#[45, 1]
    partitura_tok= self.tgt_tok_emb(partitura_tok)
    # print("partitura_tok2  : "+str(partitura_tok.shape))#[45, 1, 40]
    tgt_emb = self.positional_encoding(partitura_tok)
    # print("tgt_emb_positional  : "+str(tgt_emb.shape))#[45, 1, 40]

    mask_tgt=generate_square_subsequent_mask(tgt_emb.shape[0])
    ouput_decoder=self.transformer_decoder(tgt_emb,memory,mask_tgt)
    ouput_decoder=ouput_decoder.transpose(1,0)
    # print("ouput_decoder  : "+str(ouput_decoder.shape))#[45, N, 40]
    linear_decoder=self.fc2_linear(ouput_decoder)
    # print("linear_decoder  : "+str(linear_decoder.shape))#[45, N, 233]
    
   
  
    output_softmax = self.softmax_out(linear_decoder)
    # print("output_softmax  : "+str(output_softmax.shape))#[N, 45, 233]
    
    return  output_softmax 

  def memoria(self,x,wave_comple):
    conv2d_embedding1 = self.conv2Dblock1(x) # x == N/batch * channel * freq * time
    # print("conv2d_embedding1 : "+str(conv2d_embedding1.shape))# [N, 4, 10, 215]
    
    conv2d_embedding1 = conv2d_embedding1.reshape(conv2d_embedding1.shape[0],-1,conv2d_embedding1.shape[-1]) #[n,40,215]
    # print("conv2d_embedding1 Reshape : "+str(conv2d_embedding1.shape))#[n,40,215]
    ############ 2nd parallel Conv2D block: 4 Convolutional layers #############################
    
    conv2d_embedding2 = self.conv2Dblock2(x) # x == N/batch * channel * freq * time
    # print("conv2d_embedding2 : "+str(conv2d_embedding2.shape))# [N, 4, 10, 215]
   
    conv2d_embedding2 = conv2d_embedding2.reshape(conv2d_embedding2.shape[0],-1,conv2d_embedding2.shape[-1])#[n,40,215]
    # print("conv2d_embedding2 Reshape : "+str(conv2d_embedding2.shape))#[n,40,215]
   
    # x_maxpool = self.transformer_maxpool(x)
    # print("x_maxpool  : "+str(x_maxpool.shape))#[1, 1, 40, 430]
    
    x_maxpool_reduced = wave_comple
    # print("wave 2 : "+str(x_maxpool_reduced.shape))#[1, 40, 581]
    
    wave_comple = wave_comple.permute(2,0,1) 
    # print("x------>entrada Encoder : "+str(wave_comple.shape))#[581, 1, 40]

    transformer_output = self.transformer_encoder(wave_comple)
    # print("salida  Encoder : "+str(transformer_output.shape))#[581, 1, 40]
    transformer_output=transformer_output.permute(1,2,0) #[1, 40, 581] 
    
    #sumaTensores=conv2d_embedding1 + conv2d_embedding2 + transformer_output
    sumaTensores=torch.cat((conv2d_embedding1, conv2d_embedding2, transformer_output), 2)
    # print("sumaTensores : "+str(sumaTensores.shape))#[1, 40, 1011]
    entrada_Decoder=sumaTensores.permute(2,0,1)#[430,n,40]

    # print("entrada_Decoder : "+str(entrada_Decoder.shape))#[1011, 1, 40]
    memory=self.positional_encoding(entrada_Decoder)
    # print("memory positional  : "+str(memory.shape))#[430,n,40]
    return memory

  def decoder(self,memory,partitura_tok):
    partitura_tok=partitura_tok.transpose(1,0)#[1,45]
    # print("partitura_tok  : "+str(partitura_tok.shape))#[45, 1]
    partitura_tok= self.tgt_tok_emb(partitura_tok)
    # print("partitura_tok2  : "+str(partitura_tok.shape))#[45, 1, 40]
    tgt_emb = self.positional_encoding(partitura_tok)
    # print("tgt_emb_positional  : "+str(tgt_emb.shape))#[45, 1, 40]

    mask_tgt=generate_square_subsequent_mask(tgt_emb.shape[0])
    ouput_decoder=self.transformer_decoder(tgt_emb,memory,mask_tgt)
   
    return ouput_decoder
  


def generate_square_subsequent_mask(sz):
    DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

In [ ]:
class TmusicTrasforms2(nn.Module):
  def __init__(self,tgt_vocabulario,emb=40):
    super().__init__() 
    
    ################ TRANSFORMER BLOCK #############################
    # maxpool the input feature map/tensor to the transformer 
    # a rectangular kernel worked better here for the rectangular input spectrogram feature map/tensor
    self.transformer_maxpool = nn.MaxPool2d(kernel_size=[1,4], stride=[1,4])
    
    
    self.tgt_tok_emb = TokenEmbedding(tgt_vocabulario, emb)
    self.positional_encoding = PositionalEncoding(emb, dropout=0.1)
    
    transformer_layer = nn.TransformerEncoderLayer(
        d_model=40, # input feature (frequency) dim after maxpooling 40*1723 -> 40*430 (MFC*time)
        nhead=8, # 4 self-attention layers in each multi-head self-attention layer in each encoder block
        dim_feedforward=512, # 2 linear layers in each encoder block's feedforward network: dim 40-->512--->40
        dropout=0.4, 
        activation='relu' # ReLU: avoid saturation/tame gradient/reduce compute time
    )
    
    self.transformer_encoder = nn.TransformerEncoder(transformer_layer, num_layers=6)
    
    transformer_decoder_layer=nn.TransformerDecoderLayer(
        d_model=emb,
        nhead=8,
        dim_feedforward=512,
        dropout=0.4,
        activation='relu'
    )
    self.transformer_decoder=nn.TransformerDecoder(transformer_decoder_layer,num_layers=6)

   

   
    
    self.fc2_linear = nn.Linear(emb,tgt_vocabulario)
    
    
    self.softmax_out = nn.Softmax(dim=2) # dim==1 is the freq embedding
  
  def forward(self,x,partitura_tok,wave_comple):
    
    x_maxpool_reduced = wave_comple
    # print("wave 2 : "+str(x_maxpool_reduced.shape))#[1, 40, 581]
    
    wave_comple = wave_comple.permute(2,0,1) 
    # print("x------>entrada Encoder : "+str(wave_comple.shape))#[581, 1, 40]
    wave_comple = self.positional_encoding(wave_comple)
    transformer_output = self.transformer_encoder(wave_comple)
    # print("salida  Encoder : "+str(transformer_output.shape))#[581, 1, 40]
    
    
    
    

   
    memory=transformer_output
    # print("memory positional  : "+str(memory.shape))#[1011, 1, 40]

    partitura_tok=partitura_tok.transpose(1,0)#[1,45]
    # print("partitura_tok  : "+str(partitura_tok.shape))#[45, 1]
    partitura_tok= self.tgt_tok_emb(partitura_tok)
    # print("partitura_tok2  : "+str(partitura_tok.shape))#[45, 1, 40]
    tgt_emb = self.positional_encoding(partitura_tok)
    # print("tgt_emb_positional  : "+str(tgt_emb.shape))#[45, 1, 40]

    mask_tgt=generate_square_subsequent_mask(tgt_emb.shape[0])
    ouput_decoder=self.transformer_decoder(tgt_emb,memory,mask_tgt)
    ouput_decoder=ouput_decoder.transpose(1,0)
    # print("ouput_decoder  : "+str(ouput_decoder.shape))#[45, N, 40]
    linear_decoder=self.fc2_linear(ouput_decoder)
    # print("linear_decoder  : "+str(linear_decoder.shape))#[45, N, 233]
    
   
  
    output_softmax = self.softmax_out(linear_decoder)
    # print("output_softmax  : "+str(output_softmax.shape))#[N, 45, 233]
    
    return  output_softmax 

  def memoria(self,x,wave_comple):
    x_maxpool_reduced = wave_comple
    print("wave 2 : "+str(x_maxpool_reduced.shape))#[1, 40, 581]
    
    wave_comple = wave_comple.permute(2,0,1) 
    print("x------>entrada Encoder : "+str(wave_comple.shape))#[581, 1, 40]

    transformer_output = self.transformer_encoder(wave_comple)
    print("salida  Encoder : "+str(transformer_output.shape))#[581, 1, 40]
    
    
    
    

   
    memory=transformer_output
    return memory

  def decoder(self,memory,partitura_tok):
    partitura_tok=partitura_tok.transpose(1,0)#[1,45]
    # print("partitura_tok  : "+str(partitura_tok.shape))#[45, 1]
    partitura_tok= self.tgt_tok_emb(partitura_tok)
    # print("partitura_tok2  : "+str(partitura_tok.shape))#[45, 1, 40]
    tgt_emb = self.positional_encoding(partitura_tok)
    # print("tgt_emb_positional  : "+str(tgt_emb.shape))#[45, 1, 40]

    mask_tgt=generate_square_subsequent_mask(tgt_emb.shape[0])
    ouput_decoder=self.transformer_decoder(tgt_emb,memory,mask_tgt)
   
    return ouput_decoder
  


def generate_square_subsequent_mask(sz):
    DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

In [ ]:
class TmusicTrasforms3(nn.Module):
  def __init__(self,tgt_vocabulario,emb=512):
    super().__init__() 
    
    ################ TRANSFORMER BLOCK #############################
    # maxpool the input feature map/tensor to the transformer 
    # a rectangular kernel worked better here for the rectangular input spectrogram feature map/tensor
    self.transformer_maxpool = nn.MaxPool2d(kernel_size=[1,4], stride=[1,4])
    
    
    self.tgt_tok_emb = TokenEmbedding(tgt_vocabulario, emb)
    self.entrada_emb = TokenEmbedding(64*26, emb)
    self.positional_encoding = PositionalEncoding(emb, dropout=0.1)
    
    transformer_layer = nn.TransformerEncoderLayer(
        d_model=emb, # input feature (frequency) dim after maxpooling 40*1723 -> 40*430 (MFC*time)
        nhead=8, # 4 self-attention layers in each multi-head self-attention layer in each encoder block
        dim_feedforward=512, # 2 linear layers in each encoder block's feedforward network: dim 40-->512--->40
        dropout=0.4, 
        activation='relu' # ReLU: avoid saturation/tame gradient/reduce compute time
    )
    
    self.transformer_encoder = nn.TransformerEncoder(transformer_layer, num_layers=6)
    
    transformer_decoder_layer=nn.TransformerDecoderLayer(
        d_model=emb,
        nhead=8,
        dim_feedforward=512,
        dropout=0.4,
        activation='relu'
    )
    self.transformer_decoder=nn.TransformerDecoder(transformer_decoder_layer,num_layers=6)

    ############### 1ST PARALLEL 2D CONVOLUTION BLOCK ############
    # 3 sequential conv2D layers:  (1,40,1723) --> (8, 20, 861) -> (16, 4, 172) -> (32, 2, 86) -> (64,1,43)
    self.conv2Dblock1 = nn.Sequential(
        
       # 1st 2D convolution layer
            nn.Conv2d(
                in_channels=1, # input volume depth == input channel dim == 1
                out_channels=8, # expand output feature map volume's depth to 16
                kernel_size=3, # typical 3*3 stride 1 kernel
                stride=1,
                padding=1
                      ),
            nn.BatchNorm2d(8), # batch normalize the output feature map before activation
            nn.ReLU(), # feature map --> activation map
            nn.MaxPool2d(kernel_size=2, stride=2), #typical maxpool kernel size
            nn.Dropout(p=0.3), #randomly zero 30% of 1st layer's output feature map in training
            
            # 2nd 2D convolution layer identical to last except output dim, maxpool kernel
            nn.Conv2d(
                in_channels=8, 
                out_channels=16, # expand output feature map volume's depth to 32
                kernel_size=3,
                stride=1,
                padding=1
                      ),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4, stride=4), # increase maxpool kernel for subsequent filters
            nn.Dropout(p=0.3), 
            
            # 3rd 2D convolution layer identical to last except output dim
            nn.Conv2d(
                in_channels=16,
                out_channels=32, # expand output feature map volume's depth to 64
                kernel_size=3,
                stride=1,
                padding=1
                      ),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4, stride=4),
            nn.Dropout(p=0.3),

       
    )
    

   
    
    self.fc2_linear = nn.Linear(emb,tgt_vocabulario)
    
    
    self.softmax_out = nn.Softmax(dim=2) # dim==1 is the freq embedding
  
  def forward(self,x,partitura_tok,wave_comple):
    # print("x input : "+str(x.shape))# (1,1,40,1723)
    ############ 1st parallel Conv2D block: 4 Convolutional layers ############################
    
    conv2d_embedding1 = self.conv2Dblock1(x) # #[32,1,26]
    # print("conv2d_embedding1 : "+str(conv2d_embedding1.shape))# [N, 4, 10, 215]
    
    conv2d_embedding1 = torch.flatten(conv2d_embedding1, start_dim=1)  #[832]
    
    entrada_encoder = self.entrada_emb(conv2d_embedding1)
   
    # print("entrada_encoder  : "+str(entrada_encoder.shape))#[n,832,512]
    entrada_encoder=entrada_encoder.transpose(1,0)
    # print("entrada_encoder  : "+str(entrada_encoder.shape))
    entrada_encoder=self.positional_encoding(entrada_encoder)
    transformer_output = self.transformer_encoder(entrada_encoder)
    # print("salida  Encoder : "+str(transformer_output.shape))#[581, 1, 40]
  
    memory=transformer_output
    # print("memory positional  : "+str(memory.shape))#[1011, 1, 40]

    partitura_tok=partitura_tok.transpose(1,0)#[1,45]
    # print("partitura_tok  : "+str(partitura_tok.shape))#[45, 1]
    partitura_tok= self.tgt_tok_emb(partitura_tok)
    # print("partitura_tok2  : "+str(partitura_tok.shape))#[45, 1, 40]
    tgt_emb = self.positional_encoding(partitura_tok)
    # print("tgt_emb_positional  : "+str(tgt_emb.shape))#[45, 1, 40]

    mask_tgt=generate_square_subsequent_mask(tgt_emb.shape[0])
    ouput_decoder=self.transformer_decoder(tgt_emb,memory,mask_tgt)
    ouput_decoder=ouput_decoder.transpose(1,0)
    # print("ouput_decoder  : "+str(ouput_decoder.shape))#[45, N, 40]
    linear_decoder=self.fc2_linear(ouput_decoder)
    # print("linear_decoder  : "+str(linear_decoder.shape))#[45, N, 233]
    
   
  
    output_softmax = self.softmax_out(linear_decoder)
    # print("output_softmax  : "+str(output_softmax.shape))#[N, 45, 233]
    
    return  output_softmax 

  def memoria(self,x,wave_comple):
    conv2d_embedding1 = self.conv2Dblock1(x) # x == N/batch * channel * freq * time
    # print("conv2d_embedding1 : "+str(conv2d_embedding1.shape))# [N, 4, 10, 215]
    
    conv2d_embedding1 = conv2d_embedding1.reshape(conv2d_embedding1.shape[0],-1,conv2d_embedding1.shape[-1]) #[n,40,215]
    # print("conv2d_embedding1 Reshape : "+str(conv2d_embedding1.shape))#[n,40,215]
    ############ 2nd parallel Conv2D block: 4 Convolutional layers #############################
    
    conv2d_embedding2 = self.conv2Dblock2(x) # x == N/batch * channel * freq * time
    # print("conv2d_embedding2 : "+str(conv2d_embedding2.shape))# [N, 4, 10, 215]
   
    conv2d_embedding2 = conv2d_embedding2.reshape(conv2d_embedding2.shape[0],-1,conv2d_embedding2.shape[-1])#[n,40,215]
    # print("conv2d_embedding2 Reshape : "+str(conv2d_embedding2.shape))#[n,40,215]
   
    # x_maxpool = self.transformer_maxpool(x)
    # print("x_maxpool  : "+str(x_maxpool.shape))#[1, 1, 40, 430]
    
    x_maxpool_reduced = wave_comple
    # print("wave 2 : "+str(x_maxpool_reduced.shape))#[1, 40, 581]
    
    wave_comple = wave_comple.permute(2,0,1) 
    # print("x------>entrada Encoder : "+str(wave_comple.shape))#[581, 1, 40]

    transformer_output = self.transformer_encoder(wave_comple)
    # print("salida  Encoder : "+str(transformer_output.shape))#[581, 1, 40]
    transformer_output=transformer_output.permute(1,2,0) #[1, 40, 581] 
    
    #sumaTensores=conv2d_embedding1 + conv2d_embedding2 + transformer_output
    sumaTensores=torch.cat((conv2d_embedding1, conv2d_embedding2, transformer_output), 2)
    # print("sumaTensores : "+str(sumaTensores.shape))#[1, 40, 1011]
    entrada_Decoder=sumaTensores.permute(2,0,1)#[430,n,40]

    # print("entrada_Decoder : "+str(entrada_Decoder.shape))#[1011, 1, 40]
    memory=self.positional_encoding(entrada_Decoder)
    # print("memory positional  : "+str(memory.shape))#[430,n,40]
    return memory

  def decoder(self,memory,partitura_tok):
    partitura_tok=partitura_tok.transpose(1,0)#[1,45]
    # print("partitura_tok  : "+str(partitura_tok.shape))#[45, 1]
    partitura_tok= self.tgt_tok_emb(partitura_tok)
    # print("partitura_tok2  : "+str(partitura_tok.shape))#[45, 1, 40]
    tgt_emb = self.positional_encoding(partitura_tok)
    # print("tgt_emb_positional  : "+str(tgt_emb.shape))#[45, 1, 40]

    mask_tgt=generate_square_subsequent_mask(tgt_emb.shape[0])
    ouput_decoder=self.transformer_decoder(tgt_emb,memory,mask_tgt)
   
    return ouput_decoder
  


def generate_square_subsequent_mask(sz):
    DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

In [ ]:
 from sklearn.metrics import precision_score, accuracy_score
def make_train_step(model, criterion, optimizer):
    
  def train_step(espectro,wave_comple,partituraT,lenPartituras):
    model.train() 
    
    output_softmax = model(espectro,partituraT,wave_comple)
    salida =output_softmax #[n,45,233]
    output_softmax =output_softmax.transpose(0,1)#[45,N,233]

    part_onehot=nn.functional.one_hot(partituraT, 233)
    sumAcuraccy=0
    for i in range(partituraT.shape[0]):
      # predictions=torch.argmax(salida[i],dim=1)
      # pred=predictions.detach().cpu().numpy()
      # parti=partituraT[i].detach().cpu().numpy()
      # acuraccy=precision_score(pred,parti,average='micro')
      preds=np.argmax(salida[i].detach().cpu().numpy(), axis=1)
      acuraccy_bin=accuracy_score(preds, partituraT[i].detach().cpu().numpy())
      
      # print("acuraccy :: "+str(acuraccy))
      
      sumAcuraccy+=acuraccy_bin

    sumAcuraccy=sumAcuraccy/salida.shape[0]
    # print("acuraccy_bin: "+str(sumAcuraccy))
    
   
   
    # print("partitura t :"+str(partituraT[0].shape))
    # print("predictions :"+str(output_softmax[0].shape))
    loss=criterion(output_softmax.reshape(-1, output_softmax.shape[-1]), partituraT.reshape(-1))
  
    loss.backward()
    
    optimizer.step()
    
    # zero out gradients for next pass
    # pytorch accumulates gradients from backwards passes (convenient for RNNs)
    optimizer.zero_grad() 
    # print('***************')
    # print(loss)
    return loss.item(),sumAcuraccy
  return train_step

In [ ]:
def make_validate_fnc(model,criterion):
  def validate(espectro,partituraT,lenPartituras,wave_comple2):
      
    # don't want to update any network parameters on validation passes: don't need gradient
    # wrap in torch.no_grad to save memory and compute in validation phase: 
    with torch.no_grad(): 
        
      # set model to validation phase i.e. turn off dropout and batchnorm layers 
      model.eval()

      output_softmax = model(espectro,partituraT,wave_comple2)
      salida =output_softmax #[n,45,233]
      output_softmax =output_softmax.transpose(0,1)#[45,N,233]
   
    part_onehot=nn.functional.one_hot(partituraT, 233)
    sumAcuraccy=0
    for i in range(partituraT.shape[0]):
      preds=np.argmax(salida[i].detach().cpu().numpy(), axis=1)
      acuraccy_bin=accuracy_score(preds, partituraT[i].detach().cpu().numpy())
      
      # print("acuraccy :: "+str(acuraccy))
      
      sumAcuraccy+=acuraccy_bin

    sumAcuraccy=sumAcuraccy/salida.shape[0]
    # print("acuraccy_VAL: "+str(sumAcuraccy))
    
    loss=criterion(output_softmax.reshape(-1, output_softmax.shape[-1]), partituraT.reshape(-1))
    # print('*************** EVAL ')
    # print(loss)
    return loss.item(),sumAcuraccy
  return validate

In [ ]:
def make_save_checkpoint(): 
  def save_checkpoint(optimizer, model, epoch, filename):
    checkpoint_dict = {
        'optimizer': optimizer.state_dict(),
        'model': model.state_dict(),
        'epoch': epoch
    }
    torch.save(checkpoint_dict, filename)
  return save_checkpoint

def load_checkpoint(optimizer, model, filename):
  checkpoint_dict = torch.load(filename)
  epoch = checkpoint_dict['epoch']
  model.load_state_dict(checkpoint_dict['model'])
  if optimizer is not None:
    optimizer.load_state_dict(checkpoint_dict['optimizer'])
  return epoch,model

def load_model_predic(filename,model,device):
  checkpoint_dict = torch.load(filename,map_location=device)
  model.load_state_dict(checkpoint_dict['model'])
  return model

In [ ]:
#hiperparametros
batch_size=2
inicio_EPOCH='900'
final_EPOCH=1100
tgt_vocabulario=len(vocabPart)
SAVE_STEP=20
cargado=True

In [ ]:
from preprocess import collate_fn_nuevo, obtenerDatos
from tensorboardX import SummaryWriter
from tqdm import tqdm
from torch.utils.data import  DataLoader

In [ ]:




# set device to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} selected')

# instantiate model and move to GPU for training
model=TmusicTrasforms3(tgt_vocabulario=tgt_vocabulario)
model=model.to(device)
print('Numero de parametros entrenable: ',sum(p.numel() for p in model.parameters()) )


optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)
criterion = torch.nn.CrossEntropyLoss(ignore_index=0)

criterion = criterion.to(device)

save_checkpoint = make_save_checkpoint()


train_step = make_train_step(model, criterion, optimizer=optimizer)


validate = make_validate_fnc(model,criterion)

if cargado:
  inicio_EPOCH,model =load_checkpoint(optimizer,model,f"../checkpoint/forma4/transMusica-{inicio_EPOCH}.pkl")
else:
  inicio_EPOCH=0

def train(inicio_EPOCH,num_epochs,model):
  device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  dataset_train =obtenerDatos(pathPartituras='../datos2/partituras',pathVocabulario='../datos2/vocabulario2.json',
                          pathArchivoNPY='../datos2/archivosnumpy/train_audio_nombre.npy',pathcomple='../datos2/archivosnumpy/train_sinrecorte.npy')
    
  dataset_valid =obtenerDatos(pathPartituras='../datos2/partituras',pathVocabulario='../datos2/vocabulario2.json',
                          pathArchivoNPY='../datos2/archivosnumpy/valid_audio_nombre.npy',pathcomple='../datos2/archivosnumpy/valid_sinrecorte.npy')
  writer = SummaryWriter("runs/forma4")
  dataloader_train = DataLoader(dataset_train, batch_size=1, collate_fn=collate_fn_nuevo ,shuffle=True)
  dataloader_valid = DataLoader(dataset_valid, batch_size=1, collate_fn=collate_fn_nuevo ,shuffle=True)
  estep=0
  for epoch in range(inicio_EPOCH,num_epochs+1):      
    estep+=1
    pbar_train = tqdm(dataloader_train)
    pbar_valid = tqdm(dataloader_valid)
    loss_train=[]
    acuraccy_train=[]
    loss_valid=[]
    acuraccy_valid=[]
    for i, data in enumerate(pbar_train):
      pbar_train.set_description("Procesos train : %d"%epoch)
      partitura_tokenizada,wave_mfcc,len_partitura ,wave_comple= data
      # print(wave_mfcc.type())
      partitura_tokenizada = partitura_tokenizada.to(device)
      # wave_mfcc=wave_mfcc.float()
      wave_mfcc = wave_mfcc.to(device)
      wave_comple=wave_comple.to(device)
      
      loss,acuraccy= train_step(wave_mfcc,wave_comple,partitura_tokenizada,len_partitura)
      # print("loss :"+str(loss))
      # print('////////////////')
      loss_train.append(loss)
      acuraccy_train.append(acuraccy)
      
      if estep==1:
        writer.add_graph(model ,input_to_model=[wave_mfcc,partitura_tokenizada,wave_comple])

    for i, data2 in enumerate(pbar_valid):   
      # pbar_valid.set_description("Procesos Valid : %d"%epoch)s
      partitura_tokenizada2,wave_mfcc2,len_partitura2,wave_comple2 = data2
      partitura_tokenizada2=partitura_tokenizada2.to(device)
      # wave_mfcc=wave_mfcc.float()
      wave_mfcc2=wave_mfcc2.to(device)
      wave_comple2=wave_comple2.to(device)
      len_partitura2=len_partitura2.to(device)

      loss_val2,acuraccy_val2=validate(wave_mfcc2,partitura_tokenizada2,len_partitura2,wave_comple2)
      loss_valid.append(loss_val2)
      # print(loss_val2)
      # print('////////////////')
      acuraccy_valid.append(acuraccy_val2)
      

    final_loss_train=sum(loss_train)/len(loss_train)
    final_acuraccy_train=(sum(acuraccy_train)/len(acuraccy_train))*100
    final_loss_valid=sum(loss_valid)/len(loss_valid)
    final_acuraccy_valid=(sum(acuraccy_valid)/len(acuraccy_valid))*100
    writer.add_scalar("loss_train :",final_loss_train ,epoch)
    writer.add_scalar("acuraccy_train :",final_acuraccy_train ,epoch)
    writer.add_scalar("loss_valid :",final_loss_valid ,epoch)
    writer.add_scalar("acuraccy_valid :",final_acuraccy_valid ,epoch)

    if epoch % SAVE_STEP==0:
      checkpoint_filename = '../checkpoint/forma4/transMusica-{:03d}.pkl'.format(epoch)
      save_checkpoint(optimizer, model, epoch, checkpoint_filename)
    
    print('loss_train: '+str(final_loss_train)+', loss_valid: '+str(final_loss_valid)+' , acuraccy_train: '+str(final_acuraccy_train)+', acuraccy_valid:'+str(final_acuraccy_valid))
  writer.close()   
    


cuda:0 selected
Numero de parametros entrenable:  26342489


In [ ]:



for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

In [ ]:
train(inicio_EPOCH,final_EPOCH,model)

Procesos train : 900:   0%|          | 0/300 [00:07<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Procesos train : 901:   1%|          | 2/300 [00:00<00:18, 16.49it/s]

loss_train: 5.396339888572693, loss_valid: 5.368678914176093 , acuraccy_train: 6.275912082556336, acuraccy_valid:8.970859892431665


Procesos train : 902:   1%|          | 2/300 [00:00<00:17, 17.39it/s]

loss_train: 5.39378766854604, loss_valid: 5.368678914176093 , acuraccy_train: 6.460113160839047, acuraccy_valid:8.970859892431664


Procesos train : 903:   1%|          | 2/300 [00:00<00:17, 17.49it/s]

...

loss_train: 5.373974553743998, loss_valid: 5.434416347079807 , acuraccy_train: 8.451680792056981, acuraccy_valid:2.397126717724883


Procesos train : 1100:   1%|          | 2/300 [00:00<00:19, 15.64it/s]

loss_train: 5.377282190322876, loss_valid: 5.434416347079807 , acuraccy_train: 8.111225484420755, acuraccy_valid:2.3971267177248827


100%|██████████| 90/90 [00:19<00:00,  4.62it/s]


loss_train: 5.374629298845927, loss_valid: 5.434416347079807 , acuraccy_train: 8.392350683760716, acuraccy_valid:2.3971267177248827


In [ ]:
tensorboard --logdir runs


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
modelo =load_model_predic("../checkpoint/forma2/transMusica-300.pkl",model,device)
modelo=modelo.to(device)
# modelo

In [ ]:
ys = torch.ones(1, 1).fill_(2).type(torch.long)
ys

In [ ]:
def traslate(model,wave_mfcc,initToken,device,wave_comple,max_len=10):
  model.eval()
  
  # initToken=initToken.to(device)
  salidaMemory=model.memoria(wave_mfcc,wave_comple)
  ys = torch.ones(1, 1).fill_(initToken).type(torch.long).to(device)
  for i in range(max_len):
    # ys=ys.transpose(1,0)
    out=model.decoder(salidaMemory,ys) #[2,N,120]
    print("salida de decoder "+str(out.shape))
    out=out.transpose(1,0)
    print("viendo entrada al linear "+str(out[:, -1].shape))
    out2=model.fc2_linear(out[:, -1])
    print("salida del linear "+str(out2.shape))
    
    aplicaSoftmax=nn.Softmax(dim=2)

    otraSalida=aplicaSoftmax(out2)
    otraSalida= torch.argmax(otraSalida,dim=2)
    print("otra Salida "+str(otraSalida))
    _, next_word = torch.max(out2, dim=1)
    
    
    # print("next word "+str(next_word.item()))
    # print("next word shape"+str(next_word.shape))
    next_word=next_word.item()
    
    ys = torch.cat([ys,torch.ones(1, 1).type_as(wave_mfcc.data).fill_(next_word)], dim=1)
    print("//////////////////")
    print("ys -----> "+str(ys))
    
    print("ys -----> "+str(ys.shape))
    print("----------cambio")

def traslate2(model,wave_mfcc,initToken,device,wave_comple,max_len=10):
  with torch.no_grad():
    model.eval()

    # initToken=initToken.to(device)
    salidaMemory=model.memoria(wave_mfcc,wave_comple)
    ys = torch.ones(1, 1).fill_(initToken).type(torch.long).to(device)
    for i in range(max_len):
      # ys=ys.transpose(1,0)
      out=model.decoder(salidaMemory,ys)
      print("salida de decoder "+str(out.shape))
      out=out.transpose(1,0)
      print("viendo entrada al linear "+str(out[:, -1].shape))
      out2=model.fc2_linear(out[:, -1])
      print("salida del linear "+str(out2.shape))
      
      
      _, next_word = torch.max(out2, dim=1)
      
      
      print("next word "+str(next_word.item()))
      print("next word shape"+str(next_word.shape))
      next_word=next_word.item()
      
      ys = torch.cat([ys,torch.ones(1, 1).type_as(wave_mfcc.data).fill_(next_word)], dim=1)
      print("//////////////////")
      print("ys -----> "+str(ys))
      
      print("ys -----> "+str(ys.shape))
      print("----------cambio")

In [ ]:
dataset_train =obtenerDatos(pathPartituras='../datos2/partituras',pathVocabulario='../datos2/vocabulario2.json',
                          pathArchivoNPY='../datos2/archivosnumpy/train_audio_nombre.npy',pathcomple='../datos2/archivosnumpy/train_sinrecorte.npy')
dataloader_train = DataLoader(dataset_train, batch_size=1, collate_fn=collate_fn_nuevo ,shuffle=True )

for data in dataloader_train:
  partitura_tokenizada,wave_mfcc,len_partitura,wave_Comple = data
  wave_mfcc=wave_mfcc.to(device)
  wave_Comple=wave_Comple.to(device)
  partitura_tokenizada=partitura_tokenizada.to(device)
  print("partitura_ tokenizada //////////////// dimencion : "+str(partitura_tokenizada.shape))
  # print(partitura_tokenizada)

  # print("wave_mfcc //////////////// dimencion: "+str(wave_mfcc.shape))
  # print(wave_mfcc)
  # print("len paritura //////////////// dimencion: "+str(len_partitura.shape))
  # print(len_partitura)
  salid=modelo(wave_mfcc,partitura_tokenizada,wave_Comple)
  # print(salid)
  traslate2(modelo,wave_mfcc,1,device,wave_Comple)
  break

In [ ]:
from torchsummary import summary

# need device to instantiate model
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# instantiate model for 8 emotions and move to GPU 
model=TmusicTrasforms(tgt_vocabulario=tgt_vocabulario)
model=model.to(device)

# include input feature map dims in call to summary()
summary(model,[(1,40,1723),(50,)])


In [ ]:
# import torch
inputt = torch.randn(1,1, 40,1723, requires_grad=True)
partitura = torch.randint( 0,132, (2,100))
pato = torch.randn(1,40,430, requires_grad=True)
lenParitura = torch.randint( 30,50, (2,))
prueba = torch.randn(100,2,512, requires_grad=True)
print(inputt.shape)
print(partitura.shape)
print(lenParitura)
print(prueba.shape[0])

In [ ]:
print(pato.shape)
salida=torch.cat((pato,pato,pato))
print(salida.shape)

In [ ]:
pruebaconv2=nn.Conv2d(
            in_channels=1, # input volume depth == input channel dim == 1
            out_channels=2, # expand output feature map volume's depth to 16
            kernel_size=3, # typical 3*3 stride 1 kernel
            stride=1,
            padding=1,
            
                  )

In [ ]:
ress= pruebaconv2(inputt)
ress.shape

In [ ]:
reducir = nn.MaxPool2d(kernel_size=[2,1], stride=[2,1])
salida=reducir(inputt)
salida.shape

In [ ]:
cambiarReshampe = torch.randn(1,10, 4,430, requires_grad=True)
cambiarReshampe.shape

In [ ]:
cambiarReshampe = cambiarReshampe.reshape(cambiarReshampe.shape[0],-1,cambiarReshampe.shape[-1])
cambiarReshampe.shape

In [ ]:
cambiarReshampe = cambiarReshampe.permute(1,2,0) 
cambiarReshampe.shape

In [ ]:
model=TmusicTrasforms(tgt_vocabulario=233)
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model=model.to(DEVICE)
for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)
criterion = nn.CTCLoss()

In [ ]:
train_step=make_train_step(model,criterion,optimizer)
loss=train_step(inputt,partitura,lenParitura)
loss

In [ ]:
model.train()
ouput=model(inputt,partitura)
ouput

In [ ]:
alto=torch.max(pato)
alto

In [ ]:
emnb =nn.Embedding(133,512)

salida=emnb(partitura.long())
salida.shape

In [ ]:
pato[0]

In [ ]:
pato.shape

In [ ]:
x = pato.unsqueeze(2)
print(x.size())

x=x.expand([2,5544,512])
print(x.size())

In [ ]:
x[0]

In [ ]:
ys = torch.ones(1, 1).fill_(1).type(torch.long)
ys.shape

In [ ]:
ys = torch.cat([ys,torch.ones(1, 1).type_as().fill_(45)], dim=0)

In [ ]:
log_probs = torch.randn(1,40, 233).softmax(2).detach().requires_grad_()
log_probs = log_probs.reshape(-1, 233)
print(log_probs.shape)#salida del modelo
targets = torch.randint(0, 233, (40,), dtype=torch.long)
#targets = F.one_hot(targets, num_classes=233)
print(targets.shape)#tokens del las partituras
input_lengths = torch.full((1,), 40, dtype=torch.long)
print(input_lengths)#salida de 100 caracteristicas salida decoder
target_lengths = torch.full((1,), 40, dtype=torch.long)
print(target_lengths)#tamanio de las partituras

In [ ]:
log_probs

In [ ]:
targets.shape

In [ ]:
import torch.nn.functional as F  
criterion =nn.CrossEntropyLoss()
loss=criterion(log_probs, targets)
loss